## INSTALL

In [ ]:
!pip install --upgrade azure-cognitiveservices-vision-computervision;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 178 kB 44.2 MB/s 
     |████████████████████████████████| 41 kB 501 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##IMPORTS

In [ ]:
import os
import sys
import time
import pandas as pd

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

In [ ]:
subscription_key = 'INSERT AZURE KEY'
endpoint = 'https://visionleitura.cognitiveservices.azure.com/'
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

##FUNCTION | MS AZURE | OCR

PROJECT: XXX

OBJECT: IMAGE

ACTIVITY: OCR

ADDITIONAL: RegEX

In [ ]:
import re
def clean_ocr_text(ocr):
    """ Esta função realiza a limpeza do texto utilizando expressões regulares (RegEX) e função replace.
    This function performs text cleaning using regular expressions (RegEX) and replace function. """
    
    del_ = ['Cabo', 'Caber', 'Cab', 'Cont', 'Est',
            'ey', 'Com', 'Proc', 'Cool', 'Ext',
            'Co', 'abo', 'OPR', 'Rst', 'Fst', 'nt', 
            'ANATEL', 'Cahn', 'oi','Caha', 'Caho',
            'Cant', 'ANATE', 'ANAT', 'Et', 'Came',
            'CABO ÓPTICO', 'V TAL', 'VTAL', '\n', 
            'Caha:', 'Caha: ', 'plaqueta', 'queLa']     
            
    text = str(ocr)
    for item in del_:
        text = text.replace(str(item) , '').strip()
        text = text.replace(' ','')
        text = text.replace("&","8")
        text = text.replace("|","I")
        text = text.replace("×","X")
        text = text.replace("g","9")
        text = re.sub(r"[][().:;_\"'ـه₦-]","", text)

    return text

def extract_azure_ocr(filepath):
    """ This function returns OCR (Optical character recognition) which is the process that
    converts text in image format to text in human-readable character format.
    
    Esta função retorna OCR (Optical character recognition) que é o processo que 
    converte um texto em formato de imagem em um texto em formato de caracteres legíveis."""

    content = open(filepath, 'rb')
    try:
        read_response = computervision_client.read_in_stream(content, raw=True)
        read_operation_location = read_response.headers["Operation-Location"]
        operation_id = read_operation_location.split("/")[-1]

        while True:
            read_result = computervision_client.get_read_result(operation_id)
            if read_result.status.lower() not in ['notstarted', 'running']:
                break
            time.sleep(0.2)

        if read_result.status == OperationStatusCodes.succeeded:
            texts = ''
            for text_result in read_result.analyze_result.read_results:
                for line in text_result.lines:
                    texts = texts + ' ' + line.text
            return clean_ocr_text(texts)
        else:
            print('Azure: unexpected status (', read_result.status, ') for file:', filepath)
    except:
        print('Azure: fail to request OCR for file:', filepath)

    return ''

In [ ]:
img = '/content/drive/MyDrive/gestao_obras_2/Datasets/IMAGENS_TESTES/CROP_IMAGES/CROPPED/CROP_IMAGES/img_bot_114254201_10102022215403_predict.jpg'

In [ ]:
extract_azure_ocr(img)

'41K2P37F867PT04'